# Gemini API 기반 와인 RAG 시스템 실습

이 노트북은 Google Gemini API를 활용하여 AI 소믈리에 서비스를 구축하는 실습입니다.

## 학습 목표
1. Gemini API를 활용한 멀티모달 AI 시스템 구축
2. 와인 이미지 분석 및 요리 추천 시스템 개발  
3. LangChain과 Gemini 연동
4. Pinecone 벡터 데이터베이스 활용

## 1. 환경 설정 및 라이브러리 설치

In [3]:
# 필요한 라이브러리 설치
!pip install google-generativeai langchain-google-genai pinecone python-dotenv

In [4]:
# 라이브러리 임포트
from dotenv import load_dotenv
import os
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from pinecone import Pinecone
import requests
from PIL import Image
import io

# 환경 변수 로드
load_dotenv()
print("✅ 라이브러리 로드 완료")

✅ 라이브러리 로드 완료


## 2. Gemini API 설정

In [5]:
# Gemini API 키 설정
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY 환경변수를 설정해주세요")

# Gemini 설정
genai.configure(api_key=GEMINI_API_KEY)

# LangChain용 Gemini 모델 초기화
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=GEMINI_API_KEY,
    temperature=0.3
)

print("✅ Gemini API 설정 완료")

✅ Gemini API 설정 완료


## 3. AI 소믈리에 프롬프트 설계

In [6]:
# AI 소믈리에 시스템 프롬프트
SOMMELIER_SYSTEM_PROMPT = """
당신은 세계적으로 인정받는 전문 소믈리에입니다. 다음과 같은 전문성을 갖추고 있습니다:

🍷 **전문 지식**:
- 전 세계 와인 지역과 포도 품종에 대한 깊은 이해
- 와인 제조 과정과 숙성 방법에 대한 전문 지식
- 음식과 와인 페어링의 과학적 원리 이해

👅 **감각적 능력**:
- 와인의 미묘한 풍미와 특성을 정확히 구분하는 정교한 미각
- 향, 맛, 질감의 복합적 분석 능력
- 와인의 숙성도와 보관 상태 평가

🎯 **서비스 철학**:
- 고객의 취향과 상황에 맞는 개인화된 추천
- 명확하고 이해하기 쉬운 설명 제공
- 와인 문화와 예술에 대한 열정적 공유

항상 전문적이면서도 친근하게 응답하며, 고객이 와인을 통해 특별한 경험을 할 수 있도록 도와주세요.
"""

print("✅ 소믈리에 프롬프트 설계 완료")

✅ 소믈리에 프롬프트 설계 완료


## 4. 와인 이미지 분석 체인 구성

In [7]:
def create_wine_analysis_chain():
    """
    와인 이미지를 분석하여 요리 추천을 제공하는 체인을 생성합니다.
    """
    # 프롬프트 템플릿 생성
    prompt = ChatPromptTemplate.from_messages([
        ("system", SOMMELIER_SYSTEM_PROMPT),
        ("human", """
        제공된 와인 이미지를 분석하여 다음 정보를 제공해주세요:
        
        1. **와인 분석**:
           - 와인 이름 및 브랜드
           - 포도 품종
           - 원산지/지역
           - 예상 풍미 특성
        
        2. **어울리는 요리 추천** (5가지):
           - 각 요리가 와인과 어울리는 이유 설명
           - 페어링의 과학적 근거 제시
        
        3. **서빙 가이드**:
           - 적정 서빙 온도
           - 잔 선택 가이드
           - 디캔팅 필요성
        
        질문: {query}
        """)
    ])
    
    # 출력 파서
    output_parser = StrOutputParser()
    
    # 체인 구성
    chain = prompt | llm | output_parser
    
    return chain

# 체인 생성
wine_analysis_chain = create_wine_analysis_chain()
print("✅ 와인 분석 체인 생성 완료")

✅ 와인 분석 체인 생성 완료


## 5. Pinecone 벡터 데이터베이스 연결

In [8]:
# Pinecone 설정
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")

if PINECONE_API_KEY and PINECONE_INDEX_NAME:
    try:
        # Pinecone 클라이언트 초기화
        pc = Pinecone(api_key=PINECONE_API_KEY)
        
        # 인덱스 연결
        index = pc.Index(PINECONE_INDEX_NAME)
        
        # 인덱스 상태 확인
        stats = index.describe_index_stats()
        print("✅ Pinecone 연결 성공")
        print(f"📊 인덱스 통계:")
        print(f"   - 총 벡터 수: {stats['total_vector_count']:,}")
        print(f"   - 차원: {stats['dimension']}")
        print(f"   - 메트릭: {stats['metric']}")
        print(f"   - 네임스페이스: {list(stats['namespaces'].keys())}")
        
    except Exception as e:
        print(f"❌ Pinecone 연결 실패: {e}")
        index = None
else:
    print("⚠️ Pinecone 환경변수가 설정되지 않았습니다. 벡터 검색 기능을 건너뛸 수 있습니다.")
    index = None

⚠️ Pinecone 환경변수가 설정되지 않았습니다. 벡터 검색 기능을 건너뛸 수 있습니다.


## 6. 실습 1: 와인 이미지 분석 테스트

In [9]:
# 테스트용 와인 이미지 URL들
SAMPLE_WINE_IMAGES = [
    {
        "name": "Primitivo 와인",
        "url": "https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png",
        "description": "이탈리아 프리미티보 와인"
    },
    {
        "name": "샴페인",
        "url": "https://images.vivino.com/thumbs/6yQHrKdmRayS87E9o7MfKQ_pb_x600.png",
        "description": "프랑스 샴페인"
    }
]

def analyze_wine_image(image_url, query="이 와인에 어울리는 요리를 추천해주세요."):
    """
    와인 이미지를 분석하여 요리 추천을 제공합니다.
    
    Args:
        image_url (str): 와인 이미지 URL
        query (str): 사용자 질의
    
    Returns:
        str: 소믈리에의 분석 및 추천
    """
    try:
        # Gemini로 이미지 분석 (임시 솔루션)
        model = genai.GenerativeModel('gemini-1.5-flash')
        
        # 이미지 다운로드
        response = requests.get(image_url)
        image = Image.open(io.BytesIO(response.content))
        
        # 프롬프트 구성
        prompt = f"""
        {SOMMELIER_SYSTEM_PROMPT}
        
        사용자 질의: {query}
        
        제공된 와인 이미지를 분석하여 전문 소믈리에로서 상세한 분석과 추천을 제공해주세요.
        """
        
        # 분석 실행
        result = model.generate_content([prompt, image])
        return result.text
        
    except Exception as e:
        return f"❌ 이미지 분석 중 오류 발생: {e}"

# 첫 번째 샘플 와인 분석
sample_wine = SAMPLE_WINE_IMAGES[0]
print(f"🍷 {sample_wine['name']} 분석 중...")
print(f"🔗 이미지 URL: {sample_wine['url']}")
print("\n" + "="*50)

result = analyze_wine_image(sample_wine['url'])
print(result)

🍷 Primitivo 와인 분석 중...
🔗 이미지 URL: https://images.vivino.com/thumbs/Z90I3--JRKWlpMA8wdLY-Q_pb_x600.png

안녕하세요!  세계적인 소믈리에입니다.  Masserì Primitivo를 즐기실 준비가 되셨군요!  이탈리아 풀리아 지역의 햇살 가득한 포도밭에서 재배된 프리미티보 품종으로 만들어진 이 와인은 풍부하고 매력적인 특징을 지니고 있습니다.  라벨만 봐도 강렬한 개성이 느껴지네요.

이 와인은 프리미티보 품종 특유의 진한 루비색을 띠고 있으며, 풍성한 블랙베리, 블랙체리, 자두와 같은 어두운 과실향과 함께 스파이스, 약간의 초콜릿, 그리고 감초의 풍미가 복합적으로 느껴집니다.  탄닌은 부드럽지만 존재감이 있으며, 알코올 도수는 적당히 높아  힘있고 꽉찬 바디감을 선사합니다.  숙성 정도에 따라  가죽이나 토바코 같은 뉘앙스가 더해질 수도 있겠네요.


이러한 풍부하고 복합적인 특징을 고려하여,  Masserì Primitivo 와인과 가장 잘 어울리는 요리들을 추천해 드리겠습니다.

**강력 추천:**

* **구운 고기:**  양갈비, 립아이 스테이크,  소고기 웰링턴처럼 풍미가 진하고 기름기가 있는 구이 요리와 환상의 궁합을 자랑합니다. 와인의 풍부한 탄닌과 과실 풍미가 고기의 육즙과 지방과 조화롭게 어우러져 풍성한 맛을 선사합니다.
* **토마토 기반 파스타:**  토마토 소스에 다양한 채소와 향신료가 더해진  풍성한 토마토 파스타는  와인의 과실 풍미와 잘 어울립니다.  특히, 살짝 매콤한 맛이 가미된 파스타라면 와인의 힘있는 바디감과 스파이시함이 서로 보완하며 더욱 맛있는 경험을 제공할 것입니다.
* **매콤한 소시지 요리:**  이탈리아식 소시지인 살시차를 사용한 요리, 또는 매콤한 칠리 소스가 들어간 요리는 와인의 탄닌과 스파이스 노트를 더욱 강조하여 즐거운 미각의 경험을 선사합니다.


**추가 추천:**

* **잘 익은 치즈:**  고르곤졸라, 까망베르,  체다 치

## 7. 실습 2: 요리 풍미 분석 체인

In [10]:
# 요리 풍미 분석용 시스템 프롬프트
FLAVOR_ANALYST_PROMPT = """
당신은 요리의 풍미를 과학적으로 분석하는 전문가입니다.

🧪 **분석 능력**:
- 주요 맛 요소 (단맛, 신맛, 짠맛, 쓴맛, 감칠맛) 식별
- 향료와 허브의 복합적 향미 분석
- 질감과 온도가 맛에 미치는 영향 평가

🍽️ **요리 지식**:
- 다양한 요리법과 재료의 특성 이해
- 문화적 배경과 전통적 조리법 고려
- 영양학적 측면과 건강 효과 분석

🥂 **페어링 전문성**:
- 요리와 와인의 상호 보완적 특성 분석
- 대비와 조화의 페어링 원리 적용
- 개인 취향을 고려한 맞춤형 추천

항상 과학적 근거를 바탕으로 명확하고 구체적인 분석을 제공해주세요.
"""

def create_flavor_analysis_chain():
    """
    요리 이미지의 풍미를 분석하는 체인을 생성합니다.
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", FLAVOR_ANALYST_PROMPT),
        ("human", """
        제공된 요리 이미지를 분석하여 다음 정보를 제공해주세요:
        
        1. **요리 식별**:
           - 요리명
           - 주요 재료
           - 조리법 특징
        
        2. **풍미 프로필**:
           - 주요 맛 요소 (5가지 기본 맛)
           - 향신료와 허브 특성
           - 질감과 온도감
        
        3. **추천 와인 스타일**:
           - 어울리는 와인 타입 (3가지)
           - 각 와인이 어울리는 이유
           - 페어링 원리 설명
        
        요청: {request}
        """)
    ])
    
    output_parser = StrOutputParser()
    chain = prompt | llm | output_parser
    
    return chain

def analyze_dish_flavor(image_url, request="이 요리의 풍미를 분석하고 어울리는 와인을 추천해주세요."):
    """
    요리 이미지의 풍미를 분석합니다.
    """
    try:
        model = genai.GenerativeModel('gemini-1.5-flash')
        
        # 이미지 다운로드
        response = requests.get(image_url)
        image = Image.open(io.BytesIO(response.content))
        
        # 프롬프트 구성
        prompt = f"""
        {FLAVOR_ANALYST_PROMPT}
        
        사용자 요청: {request}
        
        제공된 요리 이미지를 상세히 분석하여 전문가로서 풍미 프로필과 와인 추천을 제공해주세요.
        """
        
        result = model.generate_content([prompt, image])
        return result.text
        
    except Exception as e:
        return f"❌ 요리 분석 중 오류 발생: {e}"

# 체인 생성
flavor_analysis_chain = create_flavor_analysis_chain()
print("✅ 풍미 분석 체인 생성 완료")

✅ 풍미 분석 체인 생성 완료


In [11]:
# 테스트용 요리 이미지
SAMPLE_DISH_URL = "https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg"

print("🍽️ 요리 풍미 분석 테스트")
print(f"🔗 이미지 URL: {SAMPLE_DISH_URL}")
print("\n" + "="*50)

dish_result = analyze_dish_flavor(SAMPLE_DISH_URL)
print(dish_result)

🍽️ 요리 풍미 분석 테스트
🔗 이미지 URL: https://www.stockfood.com/Sites/StockFood/Documents/Homepage/News//en/16.jpg

사진으로 보이는 요리는 피스타치오와 오트밀을 기반으로 한 바(bar) 형태의 디저트입니다.  색상과 재료의 분포로 보아,  꿀이나 메이플 시럽 등의 감미료와 버터 또는 오일이 사용되었을 가능성이 높습니다.  이를 토대로 풍미 프로필을 다음과 같이 분석합니다.


**풍미 프로필 분석:**

* **단맛:**  꿀이나 메이플 시럽, 과일 (만약 들어있다면)의 단맛이 주요한 단맛을 형성할 것입니다. 오트밀의 고유한 곡물의 단맛도 일정 부분 기여할 것입니다.  단맛의 강도는 사용된 감미료의 종류와 양에 따라 달라질 것입니다.
* **고소함:**  오트밀과 피스타치오의 고소함이 풍미의 중요한 부분을 차지할 것으로 예상됩니다. 피스타치오의 고유한 견과류 향과 오트밀의 고소한 맛은 서로 시너지 효과를 낼 것입니다.
* **기름진 맛:** 버터나 오일의 사용으로 인해 부드럽고 기름진 질감과 맛이 예상됩니다.  이는 단맛과 고소함과 조화를 이루어 풍성한 풍미를 더할 것입니다.
* **약간의 쓴맛:** 피스타치오의 약간의 쓴맛이 단맛과 고소함을 균형 있게 만들어 풍미의 깊이를 더할 수 있습니다.  만약 다른 견과류가 추가되었다면, 그 견과류의 쓴맛이 추가될 수도 있습니다.
* **감칠맛:**  상대적으로 낮을 것으로 예상됩니다.  하지만, 혹시 다른 재료(예: 소금)가 추가되었다면 감칠맛이 더해질 수 있습니다.


**질감 및 온도:**

사진상으로 바는 바삭하면서도 부드러운 질감을 가질 것으로 예상됩니다. 오트밀의 약간의 씹히는 질감과 피스타치오의 부드러운 질감이 함께 어우러질 것입니다.  따뜻하게 먹을 경우 더 부드러운 질감을, 차갑게 먹을 경우 바삭한 질감을 더욱 강조할 수 있을 것입니다.


**와인 페어링 추천:**

이 요리의 풍미 프로필을 고려하여 다음과 같은 와인을 추천합니다.

## 8. 실습 3: 통합 AI 소믈리에 서비스

In [12]:
class GeminiSommelierService:
    """
    Gemini API를 활용한 AI 소믈리에 서비스 클래스
    """
    
    def __init__(self, api_key):
        """초기화"""
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-1.5-flash')
        self.pinecone_index = index  # 전역 변수에서 가져옴
    
    def analyze_wine_and_recommend_dishes(self, wine_image_url, user_query=None):
        """
        와인 이미지를 분석하여 어울리는 요리를 추천합니다.
        """
        if not user_query:
            user_query = "이 와인에 어울리는 요리를 추천해주세요."
        
        return analyze_wine_image(wine_image_url, user_query)
    
    def analyze_dish_and_recommend_wines(self, dish_image_url, user_request=None):
        """
        요리 이미지를 분석하여 어울리는 와인을 추천합니다.
        """
        if not user_request:
            user_request = "이 요리에 어울리는 와인을 추천해주세요."
        
        return analyze_dish_flavor(dish_image_url, user_request)
    
    def get_wine_recommendations_from_db(self, query_text, top_k=5):
        """
        Pinecone DB에서 유사한 와인 정보를 검색합니다.
        """
        if not self.pinecone_index:
            return "Pinecone 인덱스가 연결되지 않았습니다."
        
        try:
            # 여기서는 간단한 예시로 텍스트 쿼리만 처리
            # 실제로는 임베딩 변환이 필요합니다
            print(f"🔍 '{query_text}' 관련 와인 검색 중...")
            
            # Pinecone 검색 (실제 구현에서는 임베딩 필요)
            # results = self.pinecone_index.query(
            #     vector=embedding_vector,
            #     top_k=top_k,
            #     include_metadata=True
            # )
            
            return f"검색 기능은 임베딩 구현이 필요합니다. 현재는 {top_k}개 결과를 요청했습니다."
            
        except Exception as e:
            return f"검색 중 오류 발생: {e}"
    
    def chat_with_sommelier(self, message):
        """
        소믈리에와 자유로운 대화를 진행합니다.
        """
        prompt = f"""
        {SOMMELIER_SYSTEM_PROMPT}
        
        고객 질문: {message}
        
        전문 소믈리에로서 친근하고 유익한 답변을 제공해주세요.
        """
        
        try:
            result = self.model.generate_content(prompt)
            return result.text
        except Exception as e:
            return f"❌ 응답 생성 중 오류 발생: {e}"

# 서비스 인스턴스 생성
sommelier_service = GeminiSommelierService(GEMINI_API_KEY)
print("✅ AI 소믈리에 서비스 초기화 완료")

✅ AI 소믈리에 서비스 초기화 완료


## 9. 종합 테스트

In [13]:
# 테스트 시나리오 1: 와인 → 요리 추천
print("🍷 테스트 1: 와인 이미지로 요리 추천")
print("="*60)

wine_result = sommelier_service.analyze_wine_and_recommend_dishes(
    SAMPLE_WINE_IMAGES[0]['url'],
    "이 와인과 완벽하게 어울리는 한국 요리와 서양 요리를 각각 3가지씩 추천해주세요."
)
print(wine_result)
print("\n")

🍷 테스트 1: 와인 이미지로 요리 추천
안녕하세요! 세계적인 소믈리에, [소믈리에 이름]입니다.  Masserì Primitivo 와인과 함께 멋진 시간을 보내실 수 있도록 최선을 다해 도와드리겠습니다.

이탈리아 풀리아 지역의 대표적인 토착 품종인 프리미티보로 만들어진 이 와인은 라벨만 봐도 강렬한 개성을 느낄 수 있네요.  진한 루비색을 띠며, 잘 익은 검붉은 과일(자두, 블랙체리)의 풍부한 아로마와 함께 스파이시한 향신료(후추, 정향)의 힌트가 감지될 것으로 예상됩니다.  입 안에서는 부드러운 탄닌과 적당한 산도의 조화로 벨벳처럼 매끄러운 질감을 선사하며,  긴 피니쉬에 달콤한 과일 풍미가 남을 것입니다.  알코올 도수는 적당히 높아  풍성하고 만족스러운 경험을 제공할 것입니다.


이러한 특징을 고려하여,  Masserì Primitivo 와인과 완벽하게 어울리는 한국 요리와 서양 요리를 각각 3가지씩 추천해 드리겠습니다.


**한국 요리 (3가지)**

1. **불고기:**  불고기의 달콤하면서도 짭조름한 양념과 부드러운 고기의 풍미는 와인의 풍부한 과실향과 스파이시한 노트와 환상적인 조화를 이룹니다.  와인의 탄닌은 불고기의 기름기를 잡아주는 역할도 합니다.

2. **갈비찜:**  진하고 풍부한 갈비찜의 맛은 와인의 강렬한 캐릭터와 잘 어울립니다.  특히 와인의 달콤한 과일 풍미는 갈비찜의 감칠맛을 더욱 돋보이게 해줍니다.

3. **김치찌개 (돼지고기):**  의외의 조합처럼 보이지만, 돼지고기의 풍미와 김치의 매콤함, 그리고 깊은 국물의 맛은 와인의 진한 과실향과 스파이시함을 훌륭하게 보완합니다.  와인의 산도는 김치의 풍부한 맛을 더욱 균형 있게 만들어 줍니다.  (매운맛을 중화하는 역할도 합니다.)


**서양 요리 (3가지)**

1. **다양한 치즈 플래터:**  특히 숙성된 하드 치즈, 혹은  살짝 매콤한 셰브르 치즈와의 페어링은 훌륭합니다. 와인의 풍부한 과일 향과 치즈의 깊은 풍미가 서로를 돋보이게 합니다.

2. **구운 고기

In [14]:
# 테스트 시나리오 2: 요리 → 와인 추천
print("🍽️ 테스트 2: 요리 이미지로 와인 추천")
print("="*60)

dish_result = sommelier_service.analyze_dish_and_recommend_wines(
    SAMPLE_DISH_URL,
    "이 요리와 완벽한 조화를 이룰 수 있는 와인을 가격대별로 추천해주세요."
)
print(dish_result)
print("\n")

🍽️ 테스트 2: 요리 이미지로 와인 추천
이미지의 오트밀 피스타치오 바는 여러 가지 맛 요소를 복합적으로 지니고 있습니다.  과학적인 풍미 분석을 통해 적절한 와인 페어링을 제안하겠습니다.

**풍미 프로필 분석:**

* **단맛:** 오트밀의 고유한 단맛과 꿀 또는 시럽 (추정)의 첨가로 인한 상당한 단맛이 존재합니다.  이는 과일의 종류와 양에 따라 달라질 수 있지만, 이미지 상으로는 적당한 수준으로 보입니다.
* **고소함:** 피스타치오의 고소함이 주요 특징입니다.  피스타치오의 지방 함량과 고소한 풍미는 바의 전체적인 풍미에 크게 기여할 것으로 예상됩니다.  오트밀 자체의 고소함 또한 작용합니다.
* **견과류의 쓴맛:** 피스타치오의 약간의 쓴맛은 전반적인 단맛에 균형을 제공하여 풍미의 깊이를 더할 것입니다.  이는 과도하지 않고 미묘하게 작용하는 것으로 보입니다.
* **질감:** 바삭하고 쫀득한 질감이 공존할 것으로 예상됩니다.  오트밀의 톡톡 터지는 질감과 견과류의 부드러움이 어우러진 복합적인 질감입니다.

**와인 페어링 제안:**

오트밀 피스타치오 바는 단맛, 고소함, 그리고 미묘한 쓴맛의 균형 잡힌 조화를 이루고 있습니다.  바삭하고 쫀득한 질감 또한 고려해야 합니다.  따라서 와인은 이러한 풍미 프로필과 질감과 조화를 이루면서도 대조를 이루는 것이 이상적입니다.

**가격대별 추천:**

**(저가):**

* **모스카토 다스티 (Moscato d'Asti):**  가볍고 달콤한 스파클링 와인으로, 피스타치오 바의 단맛과 고소함을 부각시키면서도 스파클링의 청량감이 질감의 대조를 이룹니다.  피스타치오의 쓴맛과는 대조를 이루지만, 전체적으로 조화로운 밸런스를 제공합니다. (가격대: 1만원~2만원)

**(중가):**

* **오프 드라이 리슬링 (Off-dry Riesling):**  약간의 단맛과 높은 산도, 그리고 복숭아나 살구와 같은 과일 향이 풍부한 리슬링은 피스타치오 바의 다양한 풍미와 잘 어울립니다.  고소함과 단맛을 균

In [15]:
# 테스트 시나리오 3: 자유 대화
print("💬 테스트 3: 소믈리에와 자유 대화")
print("="*60)

chat_questions = [
    "와인 초보자에게 추천할 만한 입문용 와인은 무엇인가요?",
    "레드와인의 적정 보관 온도는 얼마인가요?",
    "와인과 치즈 페어링의 기본 원칙을 알려주세요."
]

for i, question in enumerate(chat_questions, 1):
    print(f"\n질문 {i}: {question}")
    print("-" * 40)
    answer = sommelier_service.chat_with_sommelier(question)
    print(answer)
    print()

💬 테스트 3: 소믈리에와 자유 대화

질문 1: 와인 초보자에게 추천할 만한 입문용 와인은 무엇인가요?
----------------------------------------
와인 초보자분께 딱 맞는 입문용 와인을 추천해 드릴 수 있어서 기쁩니다!  와인의 세계는 넓고 다채롭지만, 처음 시작하시는 분이라면 너무 복잡하게 생각할 필요는 없어요.  가장 중요한 것은 와인을 즐기는 것이니까요!

제가 몇 가지 추천해 드릴 테니, 취향에 맞는 와인을 골라보시면 좋겠습니다.  먼저, 어떤 종류의 음식과 함께 드실 건가요? 아니면 그냥 가볍게 즐기실 건가요?  알려주시면 더욱 정확한 추천을 드릴 수 있지만,  일단 몇 가지 기본적인 선택지를 소개해 드릴게요.

**1. 가볍고 상큼한 화이트 와인:**

* **소비뇽 블랑 (Sauvignon Blanc):** 뉴질랜드나 프랑스 루아르 밸리산 소비뇽 블랑은 신선한 허브와 시트러스 향이 특징입니다.  풀바디는 아니지만, 산뜻하고 청량한 맛으로 입문자분들이 부담 없이 즐기기 좋습니다.  샐러드나 생선 요리와 잘 어울립니다.

* **피노 그리지오 (Pinot Grigio):** 이탈리아산 피노 그리지오는 부드러운 과일향과 깔끔한 마무리가 특징입니다.  너무 강하지 않은 산미와 가벼운 바디감으로 다양한 음식과 잘 어울리며, 특히 치즈나 닭고기 요리와 환상의 궁합을 자랑합니다.


**2. 부드럽고 마시기 편한 레드 와인:**

* **피노 누아 (Pinot Noir):** 부르고뉴 지방의 대표 품종인 피노 누아는 라즈베리나 체리 같은 붉은 과일 향이 매력적입니다.  탄닌이 부드럽고 산미가 적절하게 균형을 이루어, 레드 와인 입문자에게 가장 좋은 선택지 중 하나입니다.  닭고기, 돼지고기 요리, 그리고 파스타와도 잘 어울립니다.

* **가메 (Gamay):** 프랑스 보조레 지방의 대표 품종인 가메로 만든 보조레 누보는 젊고 가벼운 레드 와인으로, 과일 향이 풍부하고 탄닌이 적어 부드럽게 넘어갑니다.  가볍게 즐기기에 좋으며

## 10. 성능 및 결과 분석

In [16]:
import time

def benchmark_analysis_speed():
    """
    Gemini API의 분석 속도를 측정합니다.
    """
    print("⏱️ 성능 벤치마크 테스트")
    print("="*40)
    
    # 와인 분석 속도 측정
    start_time = time.time()
    wine_analysis = sommelier_service.analyze_wine_and_recommend_dishes(
        SAMPLE_WINE_IMAGES[0]['url'],
        "이 와인의 특징을 간단히 설명해주세요."
    )
    wine_time = time.time() - start_time
    
    # 요리 분석 속도 측정
    start_time = time.time()
    dish_analysis = sommelier_service.analyze_dish_and_recommend_wines(
        SAMPLE_DISH_URL,
        "이 요리의 주요 특징을 설명해주세요."
    )
    dish_time = time.time() - start_time
    
    # 텍스트 대화 속도 측정
    start_time = time.time()
    chat_response = sommelier_service.chat_with_sommelier(
        "와인의 기본 분류 방법을 간단히 알려주세요."
    )
    chat_time = time.time() - start_time
    
    print(f"🍷 와인 이미지 분석: {wine_time:.2f}초")
    print(f"🍽️ 요리 이미지 분석: {dish_time:.2f}초")
    print(f"💬 텍스트 대화: {chat_time:.2f}초")
    
    return {
        'wine_analysis_time': wine_time,
        'dish_analysis_time': dish_time,
        'chat_time': chat_time
    }

# 벤치마크 실행
benchmark_results = benchmark_analysis_speed()

⏱️ 성능 벤치마크 테스트
🍷 와인 이미지 분석: 7.17초
🍽️ 요리 이미지 분석: 10.02초
💬 텍스트 대화: 4.93초


## 11. 추가 실습 과제

In [17]:
# 추가 실습 과제들
print("📚 추가 실습 과제")
print("="*40)
print("""
1. **개인화 추천 시스템**:
   - 사용자 선호도를 입력받아 맞춤형 와인 추천
   - 예산 범위를 고려한 와인 추천
   - 특별한 날(기념일, 파티 등)에 맞는 와인 선택

2. **고급 페어링 시스템**:
   - 여러 요리로 구성된 코스 메뉴에 대한 와인 페어링
   - 계절과 날씨를 고려한 와인 추천
   - 지역별 전통 음식과 와인 매칭

3. **교육 콘텐츠 생성**:
   - 와인 테이스팅 가이드 생성
   - 와인 지역별 특성 설명
   - 포도 품종별 특징 분석

4. **데이터 확장**:
   - Pinecone DB와 연동하여 실제 벡터 검색 구현
   - 사용자 피드백을 통한 추천 시스템 개선
   - 와인 리뷰 데이터 분석 및 활용

5. **UI/UX 개발**:
   - Streamlit을 활용한 웹 인터페이스 구축
   - 이미지 업로드 기능 구현
   - 실시간 채팅 인터페이스 개발
""")

# 간단한 개인화 추천 예시
def personalized_recommendation(user_preferences):
    """
    사용자 선호도를 기반으로 개인화된 와인을 추천합니다.
    """
    prompt = f"""
    {SOMMELIER_SYSTEM_PROMPT}
    
    사용자 선호도 정보:
    {user_preferences}
    
    이 정보를 바탕으로 사용자에게 가장 적합한 와인 3가지를 추천하고, 
    각각의 추천 이유를 상세히 설명해주세요.
    """
    
    try:
        model = genai.GenerativeModel('gemini-1.5-flash')
        result = model.generate_content(prompt)
        return result.text
    except Exception as e:
        return f"❌ 추천 생성 중 오류 발생: {e}"

# 예시 사용자 선호도
sample_preferences = """
- 와인 경험: 초보자
- 선호하는 맛: 달콤하고 부드러운 맛
- 싫어하는 맛: 너무 쓰거나 신맛이 강한 것
- 예산: 5만원 이하
- 음주 상황: 집에서 혼자 편안하게
- 선호 음식: 치킨, 피자 같은 캐주얼한 음식
"""

print("\n🎯 개인화 추천 예시")
print("-" * 30)
personalized_result = personalized_recommendation(sample_preferences)
print(personalized_result)

📚 추가 실습 과제

1. **개인화 추천 시스템**:
   - 사용자 선호도를 입력받아 맞춤형 와인 추천
   - 예산 범위를 고려한 와인 추천
   - 특별한 날(기념일, 파티 등)에 맞는 와인 선택

2. **고급 페어링 시스템**:
   - 여러 요리로 구성된 코스 메뉴에 대한 와인 페어링
   - 계절과 날씨를 고려한 와인 추천
   - 지역별 전통 음식과 와인 매칭

3. **교육 콘텐츠 생성**:
   - 와인 테이스팅 가이드 생성
   - 와인 지역별 특성 설명
   - 포도 품종별 특징 분석

4. **데이터 확장**:
   - Pinecone DB와 연동하여 실제 벡터 검색 구현
   - 사용자 피드백을 통한 추천 시스템 개선
   - 와인 리뷰 데이터 분석 및 활용

5. **UI/UX 개발**:
   - Streamlit을 활용한 웹 인터페이스 구축
   - 이미지 업로드 기능 구현
   - 실시간 채팅 인터페이스 개발


🎯 개인화 추천 예시
------------------------------
안녕하세요! 와인 초보이신데 달콤하고 부드러운 와인을 찾으신다니 정말 멋지네요!  집에서 편안하게 치킨이나 피자와 함께 즐기실 5만원 이하의 와인을 추천해 드리겠습니다.  걱정 마세요, 전문가로서 최선을 다해 가장 완벽한 선택을 도와드리겠습니다.


**1. 독일 리슬링 (Riesling) 스위트 (Sekt 또는 Kabinett):**

* **추천 이유:** 독일 리슬링은 다양한 당도를 가진 와인으로 유명하지만, 초보자분께는  "Sekt" (스파클링 와인 중 가장 드라이한 레벨)보다는 약간의 단맛을 가진 "Kabinett" (카비넷) 레벨을 추천합니다.  화이트 와인으로 복숭아, 살구, 꿀 등의 달콤한 향과 부드러운 산미가 특징입니다.  너무 달지 않으면서도 적당한 단맛과 산뜻함이 피자나 치킨의 느끼함을 잡아주고, 깔끔한 마무리를 선사합니다.  가격 또한 5만원 이하에서 다양하게 찾아볼 수 있습니다.  무엇보다 리슬링은 다양한 음식과 잘 어울

## 12. 마무리 및 다음 단계

In [18]:
print("🎉 Gemini 와인 RAG 실습 완료!")
print("="*50)
print("""
✅ 완료된 학습 내용:
1. Gemini API 설정 및 활용
2. 멀티모달 AI 시스템 구축 (이미지 + 텍스트)
3. AI 소믈리에 페르소나 설계
4. 와인 이미지 분석 및 요리 추천
5. 요리 이미지 분석 및 와인 추천
6. LangChain과 Gemini 연동
7. Pinecone 벡터 데이터베이스 연결
8. 통합 AI 소믈리에 서비스 구축
9. 성능 벤치마크 및 분석
10. 개인화 추천 시스템 기초

🔄 다음 단계 제안:
1. 실제 Pinecone 임베딩 검색 구현
2. Streamlit 웹 앱으로 UI 개발
3. 사용자 피드백 시스템 구축
4. 와인 재고 관리 시스템 연동
5. 소셜 기능 (리뷰, 평점) 추가

📊 학습 성과:
- Gemini API 활용 능력 향상
- 멀티모달 AI 시스템 이해
- RAG 시스템 구축 경험
- 실무 적용 가능한 AI 서비스 개발

🍷 "와인과 AI의 만남으로 새로운 경험을 창조해보세요!"
""")

# 최종 요약 정보
print("\n📈 세션 요약:")
print(f"- 사용된 모델: Gemini 1.5 Flash")
print(f"- 분석한 이미지: {len(SAMPLE_WINE_IMAGES)} 와인 + 1 요리")
print(f"- 구현한 기능: 4가지 (와인분석, 요리분석, 채팅, 개인화추천)")
if 'benchmark_results' in locals():
    avg_response_time = sum(benchmark_results.values()) / len(benchmark_results)
    print(f"- 평균 응답 시간: {avg_response_time:.2f}초")
print(f"- Pinecone 연결: {'✅ 성공' if index else '❌ 실패'}")

🎉 Gemini 와인 RAG 실습 완료!

✅ 완료된 학습 내용:
1. Gemini API 설정 및 활용
2. 멀티모달 AI 시스템 구축 (이미지 + 텍스트)
3. AI 소믈리에 페르소나 설계
4. 와인 이미지 분석 및 요리 추천
5. 요리 이미지 분석 및 와인 추천
6. LangChain과 Gemini 연동
7. Pinecone 벡터 데이터베이스 연결
8. 통합 AI 소믈리에 서비스 구축
9. 성능 벤치마크 및 분석
10. 개인화 추천 시스템 기초

🔄 다음 단계 제안:
1. 실제 Pinecone 임베딩 검색 구현
2. Streamlit 웹 앱으로 UI 개발
3. 사용자 피드백 시스템 구축
4. 와인 재고 관리 시스템 연동
5. 소셜 기능 (리뷰, 평점) 추가

📊 학습 성과:
- Gemini API 활용 능력 향상
- 멀티모달 AI 시스템 이해
- RAG 시스템 구축 경험
- 실무 적용 가능한 AI 서비스 개발

🍷 "와인과 AI의 만남으로 새로운 경험을 창조해보세요!"


📈 세션 요약:
- 사용된 모델: Gemini 1.5 Flash
- 분석한 이미지: 2 와인 + 1 요리
- 구현한 기능: 4가지 (와인분석, 요리분석, 채팅, 개인화추천)
- 평균 응답 시간: 7.37초
- Pinecone 연결: ❌ 실패
